In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator # for minor ticks
from matplotlib import gridspec
from matplotlib import patches
from matplotlib.offsetbox import TextArea, VPacker, AnnotationBbox

import numpy as np

import scipy.stats as scstats
import scipy.special as sps

import sys
sys.path.append("../..")

from neuroprob.utils import stats, tools, neural_utils, biophysical
import neuroprob.models as mdl

dev = tools.PyTorch()

plt.style.use(['paper.mplstyle'])

### Inhomogenous Gamma process

The inhomogeneous Gamma point process likelihood with rate $\lambda(t)$ is 
\begin{equation}
    p(t_1, \ldots, t_n) = \frac{\Gamma(\alpha, \Lambda(t_1)) \Gamma(\alpha, \Lambda(t_n))}{\Gamma(\alpha)^n} \cdot \left[ \prod_{i=1}^n \lambda(t_i) \right] \cdot \left[ \prod_{i=2}^n(\Lambda(t_i) - \Lambda(t_{i-1}))^{\alpha-1} \right] \cdot e^{-(\Lambda(t_n) - \Lambda(t_1))}
\end{equation}
where $\Lambda(t) = \int_0^t \lambda(\tau) \mathrm{d}\tau$ and $\Gamma(\alpha, x)$ denotes the regularized upper incomplete Gamma function. It satisfies $\Gamma(\alpha, x) = 1 - G(\alpha, x)$ with 
\begin{equation}
G(\alpha n, \lambda T) = \frac{1}{\Gamma(n\alpha)} \int_0^T \lambda^{n\alpha} \tau^{n\alpha - 1} e^{-\lambda \tau} \cdot \mathrm{d}\tau
\end{equation}
called the regularized lower incomplete Gamma function. The Gamma count process probability is given by 
\begin{equation}
    Pr(N_T = n) = G(\alpha n, \lambda T) - G(\alpha (n+1), \lambda T)
\end{equation}
This distribution has $\mathbb{E}[N_T] = \sum_{k=1}^{\infty} G(\alpha k, \lambda T)$.

Let us consider the interspike intervals (ISIs), which are drawn from $Gamma(\tau|\alpha,\lambda)$. The derivation for $Pr(N_T = n)$ can then be seen intuitively using the fact that $\tau_1 + \tau_2 \sim Gamma(\tau|\alpha_1 + \alpha_2,\lambda)$. The first term is thus the probability that $n$ ISIs have a total sum below $T$, while the second term is the probability adding another ISI goes over the counting interval $T$. Note that except for the case $\alpha = 1$ (exponential distribution), $\mathbb{E}[\tau]^{-1} \neq \mathbb{E}[N_T]$. Hence the rate parameter appearing in $\mathbb{E}[\tau] = \frac{\alpha}{\lambda}$ cannot be interpreted as the 'rate' similar to the Poisson point process.

The occupancy normalized histograms used to provide a non-parametric estimate of place fields are equivalent to calculating the point estimate of $\mathbb{E}[N_T]$ for $T=1$, which by the above analysis shows this generally is a biased estimator of the underlying $\mathbb{E}[\tau]^{-1} = \frac{\lambda}{\alpha}$ in that bin. However, numerically the values are close over a wide range of $\alpha$. The differences are noticeable below $\alpha \approx 0.82$ when the bias is $>0.1$, with positive bias showing that naively using occupancy normalized histograms compensated by the shape overestimates the actual rate.

Additionally, we assumed $\mathbb{E}[N_{T_1}] + \mathbb{E}[N_{T_2}] = \mathbb{E}[N_{T_1+T_2}]$, which only holds approximately for Gamma count processes. We need $T \gg \frac{\alpha}{\lambda}$ in order to be able to ignore the effects of the renewal point process at the bin edges.

In [ ]:
rate = 10.0
shape = 0.5

fig = plt.figure(figsize=(8,4))
gs1 = fig.add_gridspec(nrows=1, ncols=2, wspace=0.2, bottom=0.6, top=0.95)
ax = fig.add_subplot(gs1[0, 0])
ax2 = fig.add_subplot(gs1[0, 1])


bins = np.arange(31)
prbs = stats.gamma_count_prob(bins, rate, 1.0, 1.0)
ax.plot(bins, prbs, color='r', label=r'Poisson')
shape = 0.5
prbs2 = stats.gamma_count_prob(bins, rate*shape, 1.0, shape)
ax.bar(bins, prbs2, width=1, color='cyan', edgecolor='grey', linewidth=1)
ax.set_xlim(-0.5, 30.5)
ax.set_yticks([])
ax.set_ylabel('probability')
ax.legend()

ax.text(-0.18, 1.05, 'A', transform=ax.transAxes, size=15)#, weight='bold') # string.ascii_uppercase[n]
ax.text(0.25, 0.1, r'$\alpha=\frac{1}{2}$', transform=ax.transAxes, size=12)


prbs = stats.gamma_count_prob(bins, rate, 1.0, 1.0)
ax2.plot(bins, prbs, color='r')
shape = 2.0
prbs2 = stats.gamma_count_prob(bins, rate*shape, 1.0, shape)
ax2.bar(bins, prbs2, width=1, color='cyan', edgecolor='grey', linewidth=1)
ax2.set_xlim(-0.5, 30.5)
ax2.set_yticks([])

fig.text(-0.12, -0.28, 'spike count', transform=ax2.transAxes, fontsize=12, ha='center')
ax2.text(0.25, 0.1, r'$\alpha=2$', transform=ax2.transAxes, size=12)


gs2 = fig.add_gridspec(nrows=1, ncols=3, wspace=0.4, bottom=0.05, top=0.45)
ax = fig.add_subplot(gs2[0, 0])
ax2 = fig.add_subplot(gs2[0, 1:])

# checking correspondence of ISI average to count average
eps = 5*1e-2
alpha_arr = np.linspace(0.2, 3.0, 1000)
beta = 100.0

avg_count = []
avg_invISI = []
for alpha in alpha_arr:
    h = 0
    add = 1
    n = 1
    while add > 1e-4:
        add = sps.gammainc(alpha*n, beta)
        h += add
        n += 1
    avg_count.append(h)
    avg_invISI.append(beta/alpha)
        
ax.text(-0.3, 1.0, 'B', transform=ax.transAxes, size=15)
ax.plot(alpha_arr, np.array(avg_count), label=r'$\mathbb{E}[N]$')
ax.plot(alpha_arr, np.array(avg_invISI), 'r--', label=r'$\mathbb{E}[\tau]^{-1}$')
ll = np.linspace(0, 6.9, 10)
ax.plot(np.ones_like(ll), ll, color='#dddddd')

ax.legend(frameon=False)
#ax.set_xlim(0.2, 3.0)
#ax.set_xticks([0.2, 1.0, 2.0])
#ax.set_ylim(0.0, 6.9)
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('count')



# Spike trains
ax2.text(-0.15, 1.0, 'C', transform=ax2.transAxes, size=15)

trials = 5
sim_samples = 1000
sim_tbin = 0.001 # s
rate = np.empty((trials, 1, sim_samples))

rate.fill(5.0)
dist_isi = mdl.point_process.ISI_gamma(0.25)
s = mdl.point_process.gen_IRP(dist_isi, rate, sim_tbin)
for t in range(trials):
    s_ = s[t]
    ax2.scatter(s_*sim_tbin, np.ones_like(s_)*t, marker='|', color='b', s=40, 
                label=r'$\alpha=\frac{1}{4}$')

rate.fill(20.0)
dist_isi = mdl.point_process.ISI_gamma(1)
s = mdl.point_process.gen_IRP(dist_isi, rate, sim_tbin)
for t in range(trials):
    s_ = s[t]
    ax2.scatter(s_*sim_tbin, np.ones_like(s_)*(t+trials+1), marker='|', s=40, color='k', 
                label=r'$\alpha=1$')

rate.fill(80.0)
dist_isi = mdl.point_process.ISI_gamma(4)
s = mdl.point_process.gen_IRP(dist_isi, rate, sim_tbin)
for t in range(trials):
    s_ = s[t]
    ax2.scatter(s_*sim_tbin, np.ones_like(s_)*(t+2*trials+2), marker='|', s=40, color='g', 
                label=r'$\alpha=4$')
    
ll = np.linspace(.0, 1., 10)
ax2.plot(ll, np.ones_like(ll)*(trials), color='grey')
ax2.plot(ll, np.ones_like(ll)*(2*trials+1.), color='grey')

ax2.set_xlabel('time (s)')
ax2.set_ylim(-1.5, trials*3+0.5 + 2)
ax2.set_xlim(0.0, 1.0)

ax2.set_yticks([2, 8, 14])
ax2.set_yticklabels([r'$\alpha=\frac{1}{4}$  ', r'$\alpha=1$  ', r'$\alpha=4$  '])
ax2.tick_params(axis='y', which='both', length=0)

ax2.spines['left'].set_visible(False)


#plt.savefig('output/gamma_process.svg')
#plt.savefig('output/gamma_process.pdf')
plt.show()

### Adding counts

In [ ]:
# Turn into count process
sim_tbin = 0.001
sim_samples = 1000
trials = 10000
gg = []

rate = np.empty((trials, sim_samples))

rate[:, :] = 5.0
#rate[:, :sim_samples//2] = 100.0
#rate[:, sim_samples//2:] = 0.0

mode = 'gamma'
if mode == 'gamma':
    shape = 0.5
    renew = mdl.point_process.ISI_gamma(shape)
elif mode == 'invgauss':
    mu = 1.
    lambd = 1.
    renew = mdl.point_process.ISI_invGauss(mu, lambd)
elif mode == 'lognorm':
    mu = 0.1
    sigma = 1.
    renew = mdl.point_process.ISI_logNormal(mu, sigma)
    
s = mdl.point_process.gen_IRP(renew, rate[:, None], sim_tbin)
for k in range(len(s)):
    gg.append(len(s[k]))
    
    
ggg = []
rate[:, :] = 5.0
#rate[:, :sim_samples//2] = 100.0
#rate[:, sim_samples//2:] = 0.0

mode = 'gamma'
if mode == 'gamma':
    shape = 0.5
    renew = mdl.point_process.ISI_gamma(shape)
elif mode == 'invgauss':
    mu = 1.
    lambd = 1.
    renew = mdl.point_process.ISI_invGauss(mu, lambd)
elif mode == 'lognorm':
    mu = 0.1
    sigma = 1.
    renew = mdl.point_process.ISI_logNormal(mu, sigma)
    
s = mdl.point_process.gen_IRP(renew, rate[:, None], sim_tbin)
for k in range(len(s)):
    ggg.append(len(s[k]))



In [ ]:
gggg = []
for h in range(len(gg)):
    gggg.append(gg[h]+ggg[h])

fig, ax = plt.subplots()
bins = np.arange(50)
prbs = stats.gamma_count_prob(bins, 10.0, shape, sim_tbin*sim_samples)
ax.plot(bins, prbs, 'r--', label=r'theory ($\alpha$={})'.format(shape))
ax.hist(gg, bins=np.arange(50)-0.5, density=True, label='simulation')
ax.legend()
ax.set_xlabel('Spike count')
ax.set_ylabel('Probability')
plt.show()

fig, ax = plt.subplots()
bins = np.arange(100)
prbs = stats.gamma_count_prob(bins, 5.0, shape, sim_tbin*sim_samples)
ax.plot(bins, prbs, 'r--', label=r'theory ($\alpha$={})'.format(shape))
ax.hist(ggg, bins=np.arange(100)-0.5, density=True, label='simulation')
ax.legend()
ax.set_xlabel('Spike count')
ax.set_ylabel('Probability')
plt.show()

fig, ax = plt.subplots()
bins = np.arange(100)
prbs = stats.gamma_count_prob(bins, 15.0, shape, sim_tbin*sim_samples)
ax.plot(bins, prbs, 'r--', label=r'theory ($\alpha$={})'.format(shape))
ax.hist(gggg, bins=np.arange(100)-0.5, density=True, label='simulation')
ax.legend()
ax.set_xlabel('Spike count')
ax.set_ylabel('Probability')
plt.show()